In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import *

In [3]:
dfx = pd.read_csv("/content/drive/MyDrive/EDMI/all_data.csv")
dfx.head()

,sku,product_name,monthly_sales_qty,total_sales_qty,total_comment_num,date_key
0,1656592516,惠氏（Wyeth）金装爱儿乐2段较大婴儿配方奶粉800克旗舰版 1罐,NaN,NaN,NaN,20200214.0
1,1656592516,惠氏（Wyeth）金装爱儿乐2段较大婴儿配方奶粉800克旗舰版 1罐,NaN,NaN,101.0,20200109.0
2,1656592516,惠氏（Wyeth）金装爱儿乐2段较大婴儿配方奶粉800克旗舰版 1罐,NaN,NaN,NaN,20200108.0
3,1422216617,惠氏（Wyeth）金装旗舰版幼儿乐3段1200克 婴幼儿配方奶粉 三段奶粉1200g 1盒,NaN,NaN,1298.0,20200221.0
4,1422216617,惠氏（Wyeth）金装旗舰版幼儿乐3段1200克 婴幼儿配方奶粉 三段奶粉1200g 1盒,NaN,NaN,1298.0,20200215.0


In [4]:
dfx["date_key"] = pd.to_datetime(dfx["date_key"], format="%Y%m%d")
dfx.head()

,sku,product_name,monthly_sales_qty,total_sales_qty,total_comment_num,date_key
0,1656592516,惠氏（Wyeth）金装爱儿乐2段较大婴儿配方奶粉800克旗舰版 1罐,NaN,NaN,NaN,2020-02-14
1,1656592516,惠氏（Wyeth）金装爱儿乐2段较大婴儿配方奶粉800克旗舰版 1罐,NaN,NaN,101.0,2020-01-09
2,1656592516,惠氏（Wyeth）金装爱儿乐2段较大婴儿配方奶粉800克旗舰版 1罐,NaN,NaN,NaN,2020-01-08
3,1422216617,惠氏（Wyeth）金装旗舰版幼儿乐3段1200克 婴幼儿配方奶粉 三段奶粉1200g 1盒,NaN,NaN,1298.0,2020-02-21
4,1422216617,惠氏（Wyeth）金装旗舰版幼儿乐3段1200克 婴幼儿配方奶粉 三段奶粉1200g 1盒,NaN,NaN,1298.0,2020-02-15


In [5]:
dfx.shape

(4275034, 6)

**What % of products have data "good enough" to calculate tcd-0030 metric, given some random end of the month
context examples of bad scenarios:**

- No data at all for total-comment-number <-- tc
- Only 1 observation
- More than 1 observation, but distance between now & last observation is far
- More than 1 observation, and distance between now & last observation is near
- More than 1 observation, distance between now & last observation is near AND there's an observation "close" to 30 days ago

In [6]:
# Let's find the unique sku's
skus = []
skus = dfx['sku'].unique()
print(f"Total Unique Sku Number is ", len(skus))

Total Unique Sku Number is  152722


In [7]:
from pandas.core.arrays import datetimes

skus_with_no_comment = []
total_rows_with_this_sku = []
rows_with_no_comment_under_this_sku = []
rows_with_comments_under_this_sku = []
percentage_of_no_comment = []
highest_date_with_this_sku = []
lowest_date_with_this_sku = []
max_date_with_nocom = []
min_date_with_nocom = []
last_obs_far = []
last_obs_near = []
obs_near_close_to_30days = []
last_obs_day_diffs = []



def find_data(sku, df):

  df_n = df.loc[df['sku'] == sku].sort_values('date_key')
  # date = pd.to_datetime("today").date()
  
  date = "2021-10-12"  # Format: YYYY-MM-DD
  date = datetime.strptime(date, '%Y-%m-%d').date()
  
  # if df_n['total_comment_num'].isna().sum() > 0 or df_n['total_comment_num'] == 0.0:
  if ((df_n['total_comment_num'].isna().sum() > 0) | (df_n['total_comment_num'] == 0.0)).any():

    skus_with_no_comment.append(sku)
    total_rows_with_this_sku.append(len(df_n))
    rows_with_no_comment = len(df_n[(df_n['total_comment_num'].isna()) | (df_n['total_comment_num'] == 0.0)])
    rows_with_no_comment_under_this_sku.append(rows_with_no_comment)
    rows_with_comments_under_this_sku.append(len(df_n) - rows_with_no_comment)
    percentage_of_no_comment.append((df_n['total_comment_num'].isna().sum() / len(df_n)) * 100)
    highest_date_with_this_sku.append(df_n.loc[dfx['sku'] == sku, 'date_key'].max().date())
    lowest_date_with_this_sku.append(df_n.loc[dfx['sku'] == sku, 'date_key'].min().date())
    max_date_with_nocom.append(df_n.loc[(dfx['sku'] == sku) & (dfx['total_comment_num'].isna()),  'date_key'].max().date())
    min_date_with_nocom.append(df_n.loc[(dfx['sku'] == sku) & (dfx['total_comment_num'].isna()),  'date_key'].min().date())

    last_obs_far_value = 0
    last_obs_near_value = 0
    obs_near_close_to_30days_value = 0

    """
    last_obs_far.append()
    last_obs_near.append()
    obs_near_close_to_30days.append()
    """

    # last_obs_day_diff = min(np.absolute((df_n['date_key'].dt.date - pd.to_datetime("today").date()).dt.days))
    last_obs_day_diff = min(np.absolute((df_n['date_key'].dt.date - date).dt.days))
    last_obs_day_diffs.append(last_obs_day_diff)
    
    if (len(df_n) - rows_with_no_comment) > 1:
      if last_obs_day_diff >= 45:
        last_obs_far_value = 1 # 1= True
      else:
        last_obs_near_value = 1
      
        # thirty_days_ago = datetime.today().date() - timedelta(days=30)
        thirty_days_ago = date - timedelta(days=30)

        last_obs_day_diff30 = min(np.absolute((df_n['date_key'].dt.date - thirty_days_ago).dt.days))

        if last_obs_day_diff30 <= 5:
          obs_near_close_to_30days_value = 1

    last_obs_far.append(last_obs_far_value)
    last_obs_near.append(last_obs_near_value)
    obs_near_close_to_30days.append(obs_near_close_to_30days_value)


    out = {
           "sku": skus_with_no_comment,
           "total_rows": total_rows_with_this_sku,
           "rows_with_no_comment": rows_with_no_comment_under_this_sku,
           "rows_with_comments": rows_with_comments_under_this_sku,
           "no_comment_percentage": percentage_of_no_comment,
           "last_obs_day_diff": last_obs_day_diffs,
           "last_obs_far": last_obs_far,
           "last_obs_near": last_obs_near,
           "obs_near_close_to_30days": obs_near_close_to_30days,
           "highest_date": highest_date_with_this_sku,
           "lowest_date": lowest_date_with_this_sku,
           "max_date_nocom": max_date_with_nocom,
           "min_date_nocom": min_date_with_nocom,
          }


  return pd.DataFrame(out).set_index(['sku'])


def get_data(i, f, df):

  uniques = list(set(df['sku']))

  for s in uniques[i:f]:
    dff = find_data(s, df)

  return dff

df_test = get_data(0, 10, dfx)
df_test

,total_rows,rows_with_no_comment,rows_with_comments,no_comment_percentage,last_obs_day_diff,last_obs_far,last_obs_near,obs_near_close_to_30days,highest_date,lowest_date,max_date_nocom,min_date_nocom
sku,,,,,,,,,,,,
72497496064,7,7,0,100.000000,375,0,0,0,2020-10-02,2020-08-19,2020-10-02,2020-08-19
100013965313,50,25,25,50.000000,0,0,1,0,2022-02-28,2021-09-28,2022-02-28,2021-09-28
26916683778,11,10,1,90.909091,474,0,0,0,2020-06-25,2020-01-08,2020-06-25,2020-01-08
57662242818,60,37,23,61.666667,417,1,0,0,2020-08-21,2020-01-03,2020-08-13,2020-01-03
10023432093697,1,1,0,100.000000,339,0,0,0,2020-11-07,2020-11-07,2020-11-07,2020-11-07
60879273989,2,2,0,100.000000,648,0,0,0,2020-01-03,2020-01-02,2020-01-03,2020-01-02
10074718218,199,113,86,56.783920,0,0,1,1,2022-02-22,2020-01-02,2022-02-21,2020-01-02
68684349452,6,6,0,100.000000,474,0,0,0,2020-06-25,2020-06-10,2020-06-25,2020-06-10
70223921164,1,1,0,100.000000,446,0,0,0,2020-07-23,2020-07-23,2020-07-23,2020-07-23


In [22]:
from pandas.core.arrays import datetimes

skus_with_no_comment = []
total_rows_with_this_sku = []
rows_with_no_comment_under_this_sku = []
rows_with_comments_under_this_sku = []
percentage_of_no_comment = []
highest_date_with_this_sku = []
lowest_date_with_this_sku = []
max_date_with_nocom = []
min_date_with_nocom = []
last_obs_far = []
last_obs_near = []
obs_near_close_to_30days = []

def create_dataframe():

  df1 = pd.DataFrame(list(zip(skus_with_no_comment, total_rows_with_this_sku, rows_with_no_comment_under_this_sku, 
                              rows_with_comments_under_this_sku, percentage_of_no_comment, 
                              last_obs_far, last_obs_near, obs_near_close_to_30days,
                              highest_date_with_this_sku, lowest_date_with_this_sku, 
                              max_date_with_nocom, min_date_with_nocom)), 
                     columns = ["sku", "total_rows", "rows_with_no_comment", "rows_with_comments", "no_comment_percentage",
                                "last_obs_far", "last_obs_near", "obs_near_close_to_30days", 
                                "highest_date", "lowest_date", 
                                "max_date_with_nocom", "min_date_with_nocom"])

  return df1



def find_data(sku, df):

  df_n = df.loc[df['sku'] == sku].sort_values('date_key')
  # date = pd.to_datetime("today").date()
  
  date = "2021-10-12"  # Format: YYYY-MM-DD
  date = datetime.strptime(date, '%Y-%m-%d').date()
  
  # if df_n['total_comment_num'].isna().sum() > 0 or df_n['total_comment_num'] == 0.0:
  if ((df_n['total_comment_num'].isna().sum() > 0) | (df_n['total_comment_num'] == 0.0)).any():

    skus_with_no_comment.append(sku)
    total_rows_with_this_sku.append(len(df_n))
    rows_with_no_comment = len(df_n[(df_n['total_comment_num'].isna()) | (df_n['total_comment_num'] == 0.0)])
    rows_with_no_comment_under_this_sku.append(rows_with_no_comment)
    rows_with_comments_under_this_sku.append(len(df_n) - rows_with_no_comment)
    percentage_of_no_comment.append((df_n['total_comment_num'].isna().sum() / len(df_n)) * 100)
    highest_date_with_this_sku.append(df_n.loc[dfx['sku'] == sku, 'date_key'].max().date())
    lowest_date_with_this_sku.append(df_n.loc[dfx['sku'] == sku, 'date_key'].min().date())
    max_date_with_nocom.append(df_n.loc[(dfx['sku'] == sku) & (dfx['total_comment_num'].isna()),  'date_key'].max().date())
    min_date_with_nocom.append(df_n.loc[(dfx['sku'] == sku) & (dfx['total_comment_num'].isna()),  'date_key'].min().date())

    last_obs_far_value = 0
    last_obs_near_value = 0
    obs_near_close_to_30days_value = 0

    """
    last_obs_far.append()
    last_obs_near.append()
    obs_near_close_to_30days.append()
    """

    # last_obs_day_diff = min(np.absolute((df_n['date_key'].dt.date - pd.to_datetime("today").date()).dt.days))
    last_obs_day_diff = min(np.absolute((df_n['date_key'].dt.date - date).dt.days))
    
    if (len(df_n) - rows_with_no_comment) > 1:
      if last_obs_day_diff > 45:
        last_obs_far_value = 1
      else:
        last_obs_near_value = 1
      
        # thirty_days_ago = datetime.today().date() - timedelta(days=30)
        thirty_days_ago = date - timedelta(days=30)

        last_obs_day_diff30 = min(np.absolute((df_n['date_key'].dt.date - thirty_days_ago).dt.days))

        if last_obs_day_diff30 < 5:
          obs_near_close_to_30days_value = 1

    last_obs_far.append(last_obs_far_value)
    last_obs_near.append(last_obs_near_value)
    obs_near_close_to_30days.append(obs_near_close_to_30days_value)
  

  return create_dataframe()


def get_data(i, f, df):

  skus_with_no_comment.clear()
  total_rows_with_this_sku.clear()
  rows_with_no_comment_under_this_sku.clear()
  rows_with_comments_under_this_sku.clear()
  percentage_of_no_comment.clear()
  highest_date_with_this_sku.clear()
  lowest_date_with_this_sku.clear()
  max_date_with_nocom.clear()
  min_date_with_nocom.clear()
  last_obs_far.clear()
  last_obs_near.clear()
  obs_near_close_to_30days.clear()
  

  uniques = list(set(df['sku']))

  for s in uniques[i:f]:
    dff = find_data(s, df)

  return dff


df_test1 = get_data(0, 10000, dfx)
# df_test1

In [24]:
print(len(df_test1))

9929


###### BS - 1

In [25]:
bs1 = df_test1.loc[(df_test1['rows_with_comments'] == 0)]
print(len(bs1))
bs1['sku'].to_csv('Mehedi_badscenario_1.csv', mode='a', index=False, header=False)

5911


##### BS - 2

In [26]:
bs2 = df_test1.loc[(df_test1['rows_with_comments'] == 1)]
print(len(bs2))
bs2['sku'].to_csv('Mehedi_badscenario_2.csv', mode='a', index=False, header=False)

361


###### BS - 3

In [27]:
bs3= df_test1[(df_test1['last_obs_far'] == 1)]
print(len(bs3))
bs3['sku'].to_csv('Mehedi_badscenario_3.csv', mode='a', index=False, header=False)

2108


##### BS - 4

In [28]:
bs4= df_test1[(df_test1['last_obs_near'] == 1)]
print(len(bs4))
bs4['sku'].to_csv('Mehedi_badscenario_4.csv', mode='a', index=False, header=False)

1549


##### BS -5 

In [29]:
bs5= df_test1[(df_test1['obs_near_close_to_30days'] == 1)]
print(len(bs5))
bs5['sku'].to_csv('Mehedi_badscenario_5.csv', mode='a', index=False, header=False)

466
